<a href="https://colab.research.google.com/github/44REAM/CEB-image_processing/blob/main/Diffusion_imaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Diffusion tensor imaging (DTI)
---


# Resources

- MRI image acquisition: https://www.youtube.com/playlist?list=PLIfBbrWfKu6-XPvqWM6aZhlKKOuRLIJWG
- Tensor image: https://www.youtube.com/watch?v=YxXyN2ifK8A
- Review practical use of DWI in brain image: https://www.mdpi.com/2072-6694/15/3/618

In [1]:
!git clone https://github.com/44REAM/CEB-image_processing.git
%cd /content/CEB-image_processing
!pip install dipy

Cloning into 'CEB-image_processing'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 313 (delta 101), reused 133 (delta 77), pack-reused 139
Receiving objects: 100% (313/313), 26.15 MiB | 16.09 MiB/s, done.
Resolving deltas: 100% (173/173), done.
/content/CEB-image_processing
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 11.6 MB/s eta 0:00:00


# DTI reconstruction

In [2]:
import numpy as np

from dipy.data import get_fnames
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table

import dipy.reconst.dti as dti

## Data

- DWI image: 4D image (x, y, z, sequence). (You can see the 4D image as sequences of 3D image)
- bval: factor that reflects the strength and timing of the gradients used to generate diffusion-weighted images.
- bvec: Direction of the gradient


In [3]:

# get name of the data
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data, affine = load_nifti(hardi_fname)

bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

print('data.shape (%d, %d, %d, %d)' % data.shape)

  0%|          | 0/5578 [00:00<?, ? MB/s]

  0%|          | 0/1 [00:00<?, ? MB/s]

  0%|          | 0/1 [00:00<?, ? MB/s]

data.shape (81, 106, 76, 160)


In [13]:
print(data.shape)
print(bvals.shape)
print(bvecs.shape)

(81, 106, 76, 160)
(160,)
(160, 3)


In [4]:
from dipy.segment.mask import median_otsu

maskdata, mask = median_otsu(data, vol_idx=range(10, 50), median_radius=3,
                             numpass=1, autocrop=True, dilate=2)
print('maskdata.shape (%d, %d, %d, %d)' % maskdata.shape)

maskdata.shape (71, 88, 62, 160)


## Diffusion Imaging

The simplest assumption on the diffusion is the water molecule can diffuse equally in all direction. We can write the image intensity after apply the diffusion gradient as below.

$$I = I_0 * e^{-b * D}$$

where $I_0$ is intensity of the image without magnetic gradient. $b$ is bvalue and $D$ is the diffusion coeffient.



In [5]:
tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(maskdata)

### Diffusion Tensor are symmetric



Diffusion Tensor are symmetric since x→y and y→x diffusivities should be the same. 



In [10]:
tenfit.quadratic_form[0,0,0,:,:]

array([[ 5.00006825e-10,  1.16322274e-25,  0.00000000e+00],
       [ 9.04728795e-26,  5.00006825e-10, -3.87740912e-26],
       [ 0.00000000e+00, -3.87740912e-26,  5.00006825e-10]])

In [ ]:
tensor_vals = dti.lower_triangular(tenfit.quadratic_form)

In [11]:
tensor_vals[0,0,0,:]

array([ 5.00006825e-10,  9.04728795e-26,  5.00006825e-10,  0.00000000e+00,
       -3.87740912e-26,  5.00006825e-10])